In [12]:
import pandas as pd
import cdsapi
#import netCDF4
#from netCDF4 import num2date
import numpy as np
import os
import xarray as xr
from scipy import stats
import math

In [13]:

def isFilePresent( file_name):
    file_present = os.path.isfile(file_name)
    if file_present:
        print('Alreaday present : '+file_name)
    else:
        print('file absent : '+file_name)
    return file_present

In [14]:
def download_climate_data_xarray(date, latitude, longitude, area, day_night_ind, result_df):
    
    c = cdsapi.Client()
    
    date_split = date.split("-")
    year = date_split[0]
    month = date_split[1]
    day = date_split[2]
    
    area_str = '_'.join(str(num) for num in area)
    #/Users/namratadakua/MSc - DataAnalytics/Year-2/Project/code
    #file_name = './historical_climate_data/'+year+'/weather-'+str(longitude)+'-'+str(latitude)+'-'+date+'.nc'
    file_name = '/Users/namratadakua/MSc - DataAnalytics/Year-2/Project/code/historical_climate_data/'+year+'/weather-'+str(longitude)+'-'+str(latitude)+'-'+date+'.nc'
    if not isFilePresent(file_name):
        
        time = '12:00'
        if day_night_ind == "N":
            time = '22:00'
        c.retrieve('reanalysis-era5-single-levels',
        {
            'product_type': 'reanalysis',
            'format': 'netcdf',
            'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_temperature',
            'soil_temperature_level_1', 'soil_temperature_level_2', 'soil_temperature_level_3',
            'soil_temperature_level_4', 'soil_type', 'total_precipitation', 'volumetric_soil_water_layer_1',
            'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4'
            ],
            'year': year,
            'month': month,
            'day': [str(day)],
            'time': [time],
            'area': area,
        },
        file_name)

    ds = xr.open_dataset(file_name)

    #longitude = np.mean(ds['longitude'].values)
    #latitude = np.mean(ds['latitude'].values)
    u10 = np.mean(ds['u10'].values)
    v10 = np.mean(ds['v10'].values)
    t2m = np.mean(ds['t2m'].values)
    stl1 = np.mean(ds['stl1'].values)
    stl2 = np.mean(ds['stl2'].values)
    stl3 = np.mean(ds['stl3'].values)
    stl4 = np.mean(ds['stl4'].values)
    #slt = np.mean(ds['slt'].values)
    tp = np.mean(ds['tp'].values)
    swvl1 = np.mean(ds['swvl1'].values)
    swvl2 = np.mean(ds['swvl2'].values)
    swvl3 = np.mean(ds['swvl3'].values)
    swvl4 = np.mean(ds['swvl4'].values)

    soil_mode, soil_count = stats.mode(ds['slt'].values, axis=None)

    new_row = {'longitude':longitude, 
           'latitude':latitude, 
           'date': date,
           'month': month,
           'daynight': day_night_ind,
           '10m_u_component_of_wind': u10,
           '10m_v_component_of_wind': v10,
           '2m_temperature': t2m,
           'soil_temperature_level_1': stl1,
           'soil_temperature_level_2': stl2,
           'soil_temperature_level_3': stl3,
           'soil_temperature_level_4': stl4,
           'soil_type': math.ceil(soil_mode),
           'total_precipitation': tp,
           'volumetric_soil_water_layer_1': swvl1,
           'volumetric_soil_water_layer_2': swvl2,
           'volumetric_soil_water_layer_3': swvl3,
           'volumetric_soil_water_layer_4': swvl4}
    result_df.loc[len(result_df)] = new_row
    

In [15]:
def group_fire_data_xarray(file_name):
    file_path = "./historical_fire_data/" + file_name + ".csv"
    fire_data = pd.read_csv(file_path)
    
    filtered_fire_data = fire_data.query("`confidence` >= 75 and `type` == 0")
    grouped_fire_data = filtered_fire_data.groupby(['acq_date', 'daynight']).agg({'latitude':'mean','longitude':'mean','brightness':'mean','confidence':'mean', 'frp':'mean', 'bright_t31':'mean'}).reset_index()

    #grouped_fire_data.apply(apply_on_each_group)

    result_df = pd.DataFrame({'longitude': pd.Series(dtype='float'),
                   'latitude': pd.Series(dtype='float'),
                   'date': pd.Series(dtype='str'),
                    'month': pd.Series(dtype='int'),
                   'daynight': pd.Series(dtype='str'),                   
                   '10m_u_component_of_wind': pd.Series(dtype='float'),
                   '10m_v_component_of_wind': pd.Series(dtype='float'),
                   '2m_temperature': pd.Series(dtype='float'),
                   'soil_temperature_level_1': pd.Series(dtype='float'),
                   'soil_temperature_level_2': pd.Series(dtype='float'),
                   'soil_temperature_level_3': pd.Series(dtype='float'),
                   'soil_temperature_level_4': pd.Series(dtype='float'),
                   'soil_type': pd.Series(dtype='float'),
                   'total_precipitation': pd.Series(dtype='float'),
                   'volumetric_soil_water_layer_1': pd.Series(dtype='float'),
                   'volumetric_soil_water_layer_2': pd.Series(dtype='float'),
                   'volumetric_soil_water_layer_3': pd.Series(dtype='float'),
                   'volumetric_soil_water_layer_4': pd.Series(dtype='float')})
    
    #print(grouped_fire_data)
    for modis_data_index, modis_row in grouped_fire_data.iterrows():
        #print(modis_row['acq_date'])
        date_value = modis_row['acq_date']
        confidence = modis_row['confidence']
        latitude = modis_row['latitude']
        longitude = modis_row['longitude']
        day_night_ind = modis_row['daynight']
        #print(type(confidence))
        if confidence > 75:
            north_coord = latitude+0.25
            south_coord = latitude-0.25
            west_coord = longitude-0.25
            east_coord = longitude+0.25
            area = [north_coord, west_coord, south_coord, east_coord]
            #print(area)
            download_climate_data_xarray(date_value, latitude, longitude, area, day_night_ind, result_df)
    
    result_filename = "./historical_climate_data/"+file_name+".csv"
    result_df.to_csv(result_filename)
    return result_filename
    

In [16]:
group_fire_data_xarray("canada_2010")
group_fire_data_xarray("canada_2011")
group_fire_data_xarray("canada_2012")
group_fire_data_xarray("canada_2013")
group_fire_data_xarray("canada_2014")
group_fire_data_xarray("canada_2015")
group_fire_data_xarray("canada_2016")
group_fire_data_xarray("canada_2017")
group_fire_data_xarray("canada_2018")
group_fire_data_xarray("canada_2019")
group_fire_data_xarray("canada_2020")
group_fire_data_xarray("canada_2021")
group_fire_data_xarray("canada_2022")
group_fire_data_xarray("canada_2023")

Alreaday present : /Users/namratadakua/MSc - DataAnalytics/Year-2/Project/code/historical_climate_data/2010/weather--115.8244-52.6805-2010-01-07.nc
Alreaday present : /Users/namratadakua/MSc - DataAnalytics/Year-2/Project/code/historical_climate_data/2010/weather--117.72398181818181-54.218413636363636-2010-01-08.nc
Alreaday present : /Users/namratadakua/MSc - DataAnalytics/Year-2/Project/code/historical_climate_data/2010/weather--115.76191666666666-54.21706666666666-2010-01-09.nc
Alreaday present : /Users/namratadakua/MSc - DataAnalytics/Year-2/Project/code/historical_climate_data/2010/weather--117.51993333333333-54.111266666666666-2010-01-10.nc
Alreaday present : /Users/namratadakua/MSc - DataAnalytics/Year-2/Project/code/historical_climate_data/2010/weather--115.6795-54.3921-2010-01-10.nc
Alreaday present : /Users/namratadakua/MSc - DataAnalytics/Year-2/Project/code/historical_climate_data/2010/weather--117.40415555555556-53.859-2010-01-12.nc
Alreaday present : /Users/namratadakua/MS

'./historical_climate_data/canada_2023.csv'